In [1]:

def cat(mytext):
    
    # list of keywords, add as you find new ones
    WolliRent        = ['slick','carena']
    ThePondRent      = ['harcourts','hills Prestige','hillside']
    Grocery          = ['Coles','woolworths','woolworth','tongli','WW Metro','supermarket','iga']
    Entertainment    = ['rebel','chemist','mcdonald','malatang','kmart','chatime','kogan','tea','indo'
                       ,'pondok','bread','subway','gong','chinese','kfc','mie','ciak','japanese','thai',
                       'saigon','bakery','racha','taste']
    Salary           = ['salary']
    Utilities        = ['TPG','ENERGY','AGL','Netflix','OPTUS']
    Insurance        = ['life','ins','medibank']
    Excluded         = ['Payment']
    Car              = ['Top One','AAMI','BP']
    # combine all the categories into one list called collection
    collection      = [WolliRent,ThePondRent,Grocery,Entertainment,Salary,Utilities,Insurance,Excluded,Car]


    for i in range(len(collection)):
        for k in range(len(collection[i])):
            if collection[i][k].lower() in mytext.lower():
                if i == 0:
                    cat = 'Rent - Wolli'
                elif i == 1:
                    cat = 'Rent - The Pond'
                elif i == 2:
                    cat = 'Grocery'
                elif i == 3:
                    cat = 'Entertainment'
                elif i == 4:
                    cat = 'Salary'
                elif i == 5:
                    cat = 'Utilities'
                elif i == 6:
                    cat = 'Insurance'
                elif i == 7:
                    cat = 'Excluded'
                elif i == 8:
                    cat = 'Car'   
                return cat
            
   

In [2]:
import pandas as pd
import numpy as np

EndDate = '2021-12-31'


######## input your forecast/budget number to the list below ###########
     
            # Desc         Amt  Start Date  Freq 
mylist = [['The Pond Rent'       ,650,'2021-01-12','7D'],
          ['Interest Pond WBC'   ,-889,'2021-01-22','7D'],
          ['Interest Pond CBA1'  ,-161,'2021-02-7','M'],
          ['Interest Pond CBA2'  ,-307,'2021-02-9','M'],
          ['Wolli Creek Rent'    ,450,'2021-02-06','7D'],
          ['Interest WOL CBA'    ,-311,'2021-01-20','7D'],
          ['Salary'              ,3413.17,'2021-01-27','14D'],
          ['Rent Quay St'        ,-700,'2021-01-18','7D'],
          ['Entertainment'       ,-375,'2021-01-18','7D'],
         ]

########################################################################


df = pd.DataFrame(columns=['Date','Description','Amount'])

for i in range(len(mylist)):
    date = pd.date_range(mylist[i][2],EndDate,freq=mylist[i][3])
    amount = mylist[i][1]
    desc = mylist[i][0]
    tmp = pd.DataFrame({'Date':date,'Description':desc,'Amount':amount})
    df = df.append(tmp)   
    
# add Period
df['Period'] = pd.to_datetime(df['Date']).dt.to_period('M')
#df = df.sort_values('Date')
#df['YTD'] = df['Amount'].cumsum()

Total = df[['Period','Description','Amount']]
forecast = Total.groupby(['Period','Description']).sum().reset_index()


In [3]:
forecast['YTD'] = forecast['Amount'].cumsum()

In [4]:
forecast

,Period,Description,Amount,YTD
0,2021-01,Entertainment,-750.00,-750.00
1,2021-01,Interest Pond WBC,-1778.00,-2528.00
2,2021-01,Interest WOL CBA,-622.00,-3150.00
3,2021-01,Rent Quay St,-1400.00,-4550.00
4,2021-01,Salary,3413.17,-1136.83
...,...,...,...,...
100,2021-12,Interest WOL CBA,-1555.00,8891.74
101,2021-12,Rent Quay St,-2800.00,6091.74
102,2021-12,Salary,10239.51,16331.25
103,2021-12,The Pond Rent,2600.00,18931.25


In [251]:
import pandas as pd
cba = pd.read_csv(r'C:\Users\m118954\mydsbook\Data\CSVData.csv', header=None,usecols=[0,1,2])
cba.columns=['Date','Amount','Description']


cba['Description'] = cba.apply(lambda x:cat(x['Description']),axis=1)
cba['Description'].fillna('Other',inplace=True)
# add Period
cba['Period'] = pd.to_datetime(cba['Date'],dayfirst=True).dt.to_period('M')

cba.to_csv(r'C:\Users\m118954\mydsbook\Data\CSVData2.csv')



actual = cba.groupby(['Period','Description']).sum().reset_index()



In [252]:
budget = pd.merge(actual,Forecast,on=['Period','Description'],how='left')
budget.rename(columns={'Amount_x':'Actual','Amount_y':'Forecast'})
budget['Var'] = budget['Amount_x'] - budget['Amount_y']

In [253]:
budget

,Period,Description,Amount_x,Amount_y,Var
0,2020-11,Excluded,566.28,NaN,NaN
1,2020-11,Grocery,-28.62,NaN,NaN
2,2020-11,Insurance,-32.94,NaN,NaN
3,2020-11,Other,-1.10,NaN,NaN
4,2020-12,Car,-1545.38,NaN,NaN
5,2020-12,Entertainment,-484.47,1875.0,-2359.47
6,2020-12,Excluded,4161.43,NaN,NaN
7,2020-12,Grocery,-606.41,NaN,NaN
8,2020-12,Insurance,-499.07,NaN,NaN
9,2020-12,Other,-781.51,NaN,NaN
